# Dataset Generation

In [ ]:
import random
import pandas as pd
def generate_sudoku_player_stats():
  """Generates a random Sudoku player stats sample.

  Returns:
    A dictionary containing the following keys:
      * completion_time: The time it took the player to complete the Sudoku puzzle.
      * hints: The number of hints the player used.
      * mistakes: The number of mistakes the player made.
      * is_completed: A boolean value indicating whether the player completed the Sudoku puzzle.
      * level: The player's level.
  """

  completion_time = random.randint(5, 30)
  hints = random.randint(0, 3)
  mistakes = random.randint(0, 3)
  is_completed = random.randint(0, 1)
  plevel = random.randint(1,4)
  level = plevel

  # Calculate the level.    
  if completion_time < 10  and (mistakes == 0 and hints == 0) and is_completed == 1:
    level = 4
  elif completion_time < 10  and ( mistakes != 0 or hints!= 0 ) and is_completed== 1:
    level = 3
  elif completion_time < 10  and is_completed== 0:
    level -= 1

  elif (10<=completion_time) and  (completion_time < 20 ) and (mistakes ==0 and hints ==0) and is_completed== 1:
    level = 3
  elif (10<=completion_time) and  (completion_time < 20 ) and (mistakes !=0 or hints !=0) and is_completed == 1:
    level = 2
  elif (10<=completion_time) and  (completion_time < 20 ) and is_completed == 0:
    level -= 1

  elif  (20<=completion_time) and  (completion_time < 30 ) and (mistakes ==0 and hints == 0) and is_completed ==1:
    level = 2
  elif  (20<=completion_time) and  (completion_time < 30 ) and (mistakes !=0 or hints != 0) and is_completed ==1:
    level = 1
  elif  (20<=completion_time) and  (completion_time < 30 ) and is_completed == 0:
    level -= 1

  else :
    level = 1

  return {
    "completion_time": completion_time,
    "hints": hints,
    "mistakes": mistakes,
    "is_completed": is_completed,
    "previous_level":plevel,
    "level": 1 if level<=0 else level
  }

def generate_sudoku_players_stats_dataset(num_samples):
  """Generates a dataset of Sudoku players stats samples.

  Args:
    num_samples: The number of samples to generate.

  Returns:
    A list of dictionaries, where each dictionary contains the Sudoku player stats
    for a single player.
  """

  sudoku_players_stats_dataset = []
  for i in range(num_samples):
    sudoku_players_stats_dataset.append(generate_sudoku_player_stats())
  return sudoku_players_stats_dataset

# Generate a dataset of 100k Sudoku players stats samples.
sudoku_players_stats_dataset = generate_sudoku_players_stats_dataset(1000000)

df = pd.DataFrame(sudoku_players_stats_dataset)

# Save the DataFrame to a CSV file
df.to_csv("C:\\Users\\faisa\\Documents\\Games_Section\\chess\\CSVs\\sudoku_players_stats.csv", index=False)



# Model Training

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


# Load your dataset
data = pd.read_csv("C:\\Users\\faisa\\Documents\\Games_Section\\chess\\CSVs\\sudoku_players_stats.csv")


# Split the data into features (X) and the target variable (y)
X = data[["completion_time","hints","mistakes","is_completed","previous_level"]]
y = data["level"]
# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# Fit and transform the labels
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

np.save("sudoku_encoded_labels.npy", label_encoder.classes_)


from keras.regularizers import l2

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(5,)),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(4,activation='softmax')
])


# Implement learning rate scheduling
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=10000, decay_rate=0.9, staircase=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Implement early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# Train the model
history = model.fit(
    X_train, y_train_encoded,
    epochs=100,  # Increase the number of epochs
    batch_size=64,  # Adjust batch size
    validation_data=(X_val, y_val_encoded),
    callbacks=[early_stopping],
    verbose=1  # Use verbose mode 2 for more detailed training output
)


# Save the model to a file
model.save_weights("sudoku_weights.keras")

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded)
print(f"Test Accuracy: {test_accuracy}")

c:\Users\faisa\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Epoch 1/100
12500/12500 [==============================] - 28s 2ms/step - loss: 0.2708 - accuracy: 0.9446 - val_loss: 0.1352 - val_accuracy: 0.9736
Epoch 2/100
12500/12500 [==============================] - 24s 2ms/step - loss: 0.1764 - accuracy: 0.9558 - val_loss: 0.1023 - val_accuracy: 0.9767
Epoch 3/100
12500/12500 [==============================] - 38s 3ms/step - loss: 0.1602 - accuracy: 0.9588 - val_loss: 0.0913 - val_accuracy: 0.9733
Epoch 4/100
12500/12500 [==============================] - 54s 4ms/step - loss: 0.1487 - accuracy: 0.9616 - val_loss: 0.0976 - val_accuracy: 0.9730
Epoch 5/100
12500/12500 [==============================] - 47s 4ms/step - loss: 0.1433 - accuracy: 0.9619 - val_loss: 0.0752 - val_accuracy: 0.9833
Epoch 6/100
12500/12500 [==============================] - 31s 3ms/step - loss: 0.1360 - accuracy: 0.9628 - val_loss: 0.0815 - val_accuracy: 0.9819
Epoch 7/100
12500/12500 [==============================] - 23s 2ms/step - loss: 0.1301 - accuracy: 0.9638 - val_

# Model Usage

In [17]:
import tensorflow as tf
from keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder
import numpy as np

weights_path = {
    "sudoku":"sudoku_weights.keras",
    "missing_words":"missing_words_weights.keras",
    "puzzle":"puzzle_weights.keras" }
classes_path = {
    "sudoku":"sudoku_encoded_labels.npy",
    "missing_words":"missing_words_encoded_labels.npy",
    "puzzle":"puzzle_encoded_labels.npy" }


def predict(features,game):
    """
    features[0] = completion_time 
    features[1] = hints 
    features[2] = mistakes 
    features[3] = is_completed 
    features[4] = previous_level 
    """
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(5,)),
        tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(4,activation='softmax')
    ])
    model.load_weights(weights_path[game])
    features = np.array([features])
    predicted_probs = model.predict(features)
    predicted_class = np.argmax(predicted_probs)
    class_labels = np.load(classes_path[game])
    predicted_level = class_labels[predicted_class]
    return predicted_level


print(predict([5,0,1,1,3],'sudoku'))


1/1 [==============================] - 0s 85ms/step
3


# FLASK

In [1]:
from flask import Flask, request, jsonify
import tensorflow as tf
from keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder
import numpy as np

weights_path = {
    "sudoku": "sudoku_weights.keras",
    "missing_words": "missing_words_weights.keras",
    "puzzle": "puzzle_weights.keras"
}
classes_path = {
    "sudoku": "sudoku_encoded_labels.npy",
    "missing_words": "missing_words_encoded_labels.npy",
    "puzzle": "puzzle_encoded_labels.npy"
}


def predict(features, game):
    """
    features[0] = completion_time
    features[1] = hints
    features[2] = mistakes
    features[3] = is_completed
    features[4] = previous_level
    """

    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(5,)),
        tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(4, activation='softmax')
    ])
    model.load_weights(weights_path[game])
    features = np.array([features])
    predicted_probs = model.predict(features)
    predicted_class = np.argmax(predicted_probs)
    class_labels = np.load(classes_path[game])
    predicted_level = class_labels[predicted_class]
    return predicted_level


app = Flask(__name__)


@app.route("/predict", methods=["POST"])
def predict_level():
    """
    Endpoint to predict the next level for a given set of features.

    Request body:
        {
            "features": [
                    completion_time, 
                    hints,
                    mistakes,
                    is_completed,
                    previous_level
                  ],


                  
            "game": "sudoku" | "missing_words" | "puzzle" , "connect4" , "chess" , "dots_boxes" , "memory_card" ,  
        }

    Response body:
        {
            "predicted_level": <predicted_level>
        }
    """

    features = request.get_json()["features"]
    game = request.get_json()["game"]

    predicted_level = predict(features, game)
    
    predicted_level_string = str(predicted_level)
    
    return jsonify({"predicted_level": predicted_level_string})


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

c:\Users\faisa\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.5:5000/ (Press CTRL+C to quit)


1/1 [==============================] - 0s 203ms/step


192.168.1.6 - - [10/Nov/2023 22:32:58] "POST /predict HTTP/1.1" 200 -


...

192.168.1.6 - - [10/Nov/2023 23:49:24] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 71ms/step


192.168.1.6 - - [10/Nov/2023 23:50:10] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 71ms/step


192.168.1.6 - - [10/Nov/2023 23:53:55] "POST /predict HTTP/1.1" 200 -
